In [1]:
import torch
import pickle

from load_data import get_data
from DataLoader import WeatherDL

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
data = get_data()
min_max = pickle.load(open('./min_max_test.pkl', 'rb'))
train = WeatherDL(
    data,
    time=slice('2000', '2002'),
    temporal_resolution=3,
    # window=84,
    min=min_max['min'],
    max=min_max['max'],
    batch_size=2,
    num_workers=6,
    persistent_workers=True,
    prefetch_factor=2,
    # multiprocessing_context='fork'
    )
# min, max = train.data_wrapper.getMinMaxValues()
valid = WeatherDL(
    data,
    time='2003',
    temporal_resolution=3,
    min=min_max['min'],
    max=min_max['max'],
    batch_size=5,
    num_workers=6,
    persistent_workers=True,
    prefetch_factor=2,
    # multiprocessing_context='fork'
    )
device


device(type='cuda', index=0)

In [2]:
input_size = train.spatio_temporal_dataset.n_channels   # n channel
n_nodes = train.spatio_temporal_dataset.n_nodes         # n nodes
horizon = train.spatio_temporal_dataset.horizon         # n prediction time steps
hidden_size = 256
batch = next(iter(train.data_loader))
batch

StaticBatch(
  input=(x=[b=3, t=112, n=2048, f=65], edge_index=[2, e=19328], edge_weight=[e=19328]),
  target=(y=[b=3, t=40, n=2048, f=65]),
  has_mask=False
)

In [3]:
from models.SynapticAttention import SynapticAttention
model = SynapticAttention(input_size=input_size, n_nodes=n_nodes, hidden_size=hidden_size, horizon=horizon)
model.to(device)
batch.to(device)
res = model(*batch.input)

skip torch.Size([3, 112, 2048, 256]), out torch.Size([1, 112, 1, 1])
skip torch.Size([3, 1, 2048, 256]), out torch.Size([3, 112, 2048, 256])


OutOfMemoryError: CUDA out of memory. Tried to allocate 672.00 MiB. GPU 0 has a total capacty of 23.43 GiB of which 140.38 MiB is free. Including non-PyTorch memory, this process has 22.86 GiB memory in use. Of the allocated memory 21.46 GiB is allocated by PyTorch, and 968.65 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
import torch.nn as nn
from tsl.nn.layers import NodeEmbedding, DiffConv, GATConv
from einops import rearrange
import snntorch as snn
from snntorch import functional as SF
encoder = nn.Linear(in_features=input_size, out_features=hidden_size)
embeddings = NodeEmbedding(n_nodes=n_nodes, emb_size=hidden_size)

encoded = encoder(batch.input.x)
emb = encoded + embeddings()

In [ ]:
torch.zeros(1, emb.size(1), 1, 1).size()

In [ ]:
res = GATConv(in_channels=hidden_size, out_channels=32)(emb[:, 0, :, :], batch.input.edge_index, batch.input.edge_weight)

In [ ]:
res[0].size()

In [ ]:
emb.size()

In [ ]:
b, t, n, f = emb.size()

In [ ]:
emb[:, 0, :, :].size()

In [ ]:
# from typing import Optional, Tuple, Union

# import torch
# import torch.nn.functional as F
# from torch import Tensor
# from torch.nn import Parameter
# from torch_geometric.nn.conv import MessagePassing
# from torch_geometric.nn.dense.linear import Linear
# from torch_geometric.nn.inits import glorot, zeros
# from torch_geometric.typing import Adj, OptPairTensor, OptTensor
# from torch_geometric.utils import add_self_loops, remove_self_loops
# from torch_sparse import SparseTensor, set_diag

# from tsl.nn.functional import sparse_softmax

# import snntorch as snn


# class GSATConv(MessagePassing):
#     def __init__(
#         self,
#         in_channels: Union[int, Tuple[int, int]],
#         out_channels: int,
#         heads: int = 1,
#         concat: bool = True,
#         dim: int = -2,
#         negative_slope: float = 0.2,
#         dropout: float = 0.0,
#         add_self_loops: bool = True,
#         edge_dim: Optional[int] = None,
#         fill_value: Union[float, Tensor, str] = 'mean',
#         bias: bool = True,
#         **kwargs,
#     ):
#         kwargs.setdefault('aggr', 'add')
#         super().__init__(node_dim=dim, **kwargs)

#         self.in_channels = in_channels
#         self.out_channels = out_channels
#         self.heads = heads
#         self.concat = concat
#         self.negative_slope = negative_slope
#         self.dropout = dropout
#         self.add_self_loops = add_self_loops
#         self.edge_dim = edge_dim
#         self.fill_value = fill_value

#         if self.concat:
#             self.head_channels = self.out_channels // self.heads
#             assert self.head_channels * self.heads == self.out_channels, \
#                 "`out_channels` must be divisible by `heads`."
#         else:
#             self.head_channels = self.out_channels

#         # In case we are operating in bipartite graphs, we apply separate
#         # transformations 'lin_src' and 'lin_dst' to source and target nodes:
#         # if isinstance(in_channels, int):
#         self.lin_src = snn.Synaptic(
#             alpha=0.9,
#             beta=0.8,
#             learn_alpha=True,
#             learn_beta=True,
#             learn_threshold=True,
#             # init_hidden=True,
#             # output=True
#         )
#         # Linear(in_channels,
#         #                         heads * self.head_channels,
#         #                         bias=False,
#         #                         weight_initializer='glorot')
#         self.lin_dst = self.lin_src
#         # else:
#         #     self.lin_src = Linear(in_channels[0],
#         #                           heads * self.head_channels,
#         #                           False,
#         #                           weight_initializer='glorot')
#         #     self.lin_dst = Linear(in_channels[1],
#         #                           heads * self.head_channels,
#         #                           False,
#         #                           weight_initializer='glorot')

#         # The learnable parameters to compute attention coefficients:
#         self.att_src = Parameter(torch.Tensor(1, heads, self.head_channels))
#         self.att_dst = Parameter(torch.Tensor(1, heads, self.head_channels))

#         if edge_dim is not None:
#             self.edge_synaptic = snn.Synaptic(
#                 alpha=0.9,
#                 beta=0.8,
#                 learn_alpha=True,
#                 learn_beta=True,
#                 learn_threshold=True,
#                 # init_hidden=True,
#                 # output=True,
#             )
#             self.lin_edge = Linear(edge_dim,
#                                    heads * self.head_channels,
#                                    bias=False,
#                                    weight_initializer='glorot')
#             self.att_edge = Parameter(
#                 torch.Tensor(1, heads, self.head_channels))
#         else:
#             self.lin_edge = None
#             self.register_parameter('att_edge', None)

#         if bias and concat:
#             self.bias = Parameter(torch.Tensor(heads * self.head_channels))
#         elif bias and not concat:
#             self.bias = Parameter(torch.Tensor(out_channels))
#         else:
#             self.register_parameter('bias', None)

#         self.reset_parameters()

#     def reset_parameters(self):
#         # self.lin_src.reset_parameters()
#         # self.lin_dst.reset_parameters()
#         if self.lin_edge is not None:
#             self.lin_edge.reset_parameters()
#         glorot(self.att_src)
#         glorot(self.att_dst)
#         glorot(self.att_edge)
#         zeros(self.bias)

#     def forward(self,
#                 x: Union[Tensor, OptPairTensor],
#                 edge_index: Adj,
#                 edge_attr: OptTensor = None,
#                 need_weights: bool = False):
#         """"""
#         node_dim = self.node_dim
#         self.node_dim = (node_dim + x.dim()) if node_dim < 0 else node_dim
#         b, t, n, c = x.size()

#         N, H, C = n, self.heads, self.head_channels

#         syn, mem = self.lin_src.init_synaptic()
#         if self.edge_synaptic is not None:
#             syn_e, membrane_e = self.edge_synaptic.init_synaptic()
#         # syn, membrane_pot = synaptic.init_synaptic()
#         # We first transform the input node features. If a tuple is passed, we
#         # transform source and target node features via separate weights:
#         # if isinstance(x, Tensor):
#         # print("x size", x.size())
#         for timestep in range(t):
#             data_at_time = x[:, timestep, : , :]
#             spike, syn, mem = self.lin_src(data_at_time, syn, mem)
#             x_src = x_dst = mem.view(*data_at_time.shape[:-1], H, C)
#             # else:  # Tuple of source and target node features:
#             #     x_src, x_dst = x
#             #     x_src = self.lin_src(x_src).view(*x_src.shape[:-1], H, C)
#             #     if x_dst is not None:
#             #         x_dst = self.lin_dst(x_dst).view(*x_dst.shape[:-1], H, C)

#             x_node_features = (x_src, x_dst)

#             # Next, we compute node-level attention coefficients, both for source
#             # and target nodes (if present):
#             alpha_src = (x_src * self.att_src).sum(dim=-1)
#             alpha_dst = None if x_dst is None else (x_dst * self.att_dst).sum(-1)
#             alpha = (alpha_src, alpha_dst)

#             if self.add_self_loops:
#                 if isinstance(edge_index, Tensor):
#                     edge_index, edge_attr = remove_self_loops(
#                         edge_index, edge_attr)
#                     edge_index, edge_attr = add_self_loops(
#                         edge_index,
#                         edge_attr,
#                         fill_value=self.fill_value,
#                         num_nodes=N)
#                 elif isinstance(edge_index, SparseTensor):
#                     if self.edge_dim is None:
#                         edge_index = set_diag(edge_index)
#                     else:
#                         raise NotImplementedError(
#                             "The usage of 'edge_attr' and 'add_self_loops' "
#                             "simultaneously is currently not yet supported for "
#                             "'edge_index' in a 'SparseTensor' form")

#             # edge_updater_type: (alpha: OptPairTensor, edge_attr: OptTensor)
#             alpha, syn_e, membrane_e = self.edge_updater(edge_index, alpha=alpha, edge_attr=edge_attr, syn_e=syn_e, membrane_e=membrane_e)

#             # propagate_type: (x: OptPairTensor, alpha: Tensor)
#             out = self.propagate(edge_index, x=x_node_features, alpha=alpha, size=(N, N))

#             if self.concat:
#                 out = out.view(*out.shape[:-2], self.out_channels)
#             else:
#                 out = out.mean(dim=-2)

#             if self.bias is not None:
#                 out += self.bias

#             if need_weights:
#                 # alpha rearrange: [... e ... h] -> [e ... h]
#                 alpha = torch.movedim(alpha, self.node_dim, 0)
#                 if isinstance(edge_index, Tensor):
#                     alpha = (edge_index, alpha)
#                 elif isinstance(edge_index, SparseTensor):
#                     alpha = edge_index.set_value(alpha, layout='coo')
#             else:
#                 alpha = None

#             self.node_dim = node_dim

#         return out, alpha

#     def edge_update(self, alpha_j: Tensor, alpha_i: OptTensor,
#                     edge_attr: OptTensor, index: Tensor, ptr: OptTensor,
#                     size_i: Optional[int], membrane_e=None, syn_e=None) -> Tensor:
#         """"""
#         # Given edge-level attention coefficients for source and target nodes,
#         # we simply need to sum them up to "emulate" concatenation:
#         alpha = alpha_j if alpha_i is None else alpha_j + alpha_i

#         if edge_attr is not None:
#             if edge_attr.dim() == 1:
#                 edge_attr = edge_attr.view(-1, 1)
#             assert self.lin_edge is not None
#             # edge_attr = self.lin_edge(edge_attr)
#             spike, syn, mem = self.edge_synaptic(edge_attr, syn_e, membrane_e)
#             edge_attr = self.lin_edge(mem)
#             edge_attr = edge_attr.view(-1, self.heads, self.head_channels)
#             alpha_edge = (edge_attr * self.att_edge).sum(dim=-1)
#             shape = [1] * (alpha.ndim - 1) + [self.heads]
#             shape[self.node_dim] = alpha_edge.size(0)
#             alpha = alpha + alpha_edge.view(shape)

#         alpha = F.leaky_relu(alpha, self.negative_slope)
#         alpha = sparse_softmax(alpha,
#                                index,
#                                num_nodes=size_i,
#                                ptr=ptr,
#                                dim=self.node_dim)
#         alpha = F.dropout(alpha, p=self.dropout, training=self.training)
#         return alpha, syn, mem

#     def message(self, x_j: Tensor, alpha: Tensor) -> Tensor:
#         """"""
#         return alpha.unsqueeze(-1) * x_j

#     def __repr__(self) -> str:
#         return (f'{self.__class__.__name__}({self.in_channels}, '
#                 f'{self.out_channels}, heads={self.heads})')


In [ ]:
spatial = DiffConv(hidden_size, out_channels=hidden_size, k=2, activation='relu')
emn_s = spatial(emb, batch.input.edge_index, batch.input.edge_weight)

In [ ]:
stacked = torch.cat((emb, emn_s), dim=-1)
stacked.size()

In [ ]:
from models.layers.GSATConv import GSATConv
res = GSATConv(in_channels=(1 * hidden_size) + hidden_size, out_channels=hidden_size, dim = 1, edge_dim=1)(stacked, batch.input.edge_index, batch.input.edge_weight)

In [ ]:
res[0].size()

In [ ]:
Parameter(torch.Tensor(1, 4, 64))

In [ ]:
beta = 0.9
rlif = snn.RLeaky(beta=beta, linear_features=hidden_size)
spike, membrane_pot = rlif.init_rleaky()

In [ ]:
from snntorch import surrogate
spike_grad=surrogate.atan(alpha=2.0)
thresh=1
l = snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True,threshold=thresh, output=True)

In [ ]:
torch.nn.Parameter(data=torch.tensor(0.9), requires_grad=True)

In [ ]:
def get_multiplier(i):
    return max((i * 2), 1)
    # return i + 1

In [ ]:
for i in range(3):
    print(f"size {emb.size(-1) * get_multiplier(i)}")

In [ ]:
# alpha = torch.nn.Parameter(data=torch.tensor(0.9), requires_grad=True)
# beta = torch.nn.Parameter(data=torch.tensor(0.9), requires_grad=True)
synaptic = snn.Synaptic(
    alpha=torch.Tensor(emb.size(-1)),
    beta=torch.Tensor(emb.size(-1)),
    learn_alpha=True,
    learn_beta=True,
    learn_threshold=True,
    init_hidden=True
    )
# syn, membrane_pot = synaptic.init_synaptic()

synaptic2 = snn.Synaptic(
    # alpha=torch.Tensor(emb.size(-1) * 2),
    # beta=torch.Tensor(emb.size(-1) * 2),
    alpha=torch.Tensor(emb.size(-1)),
    beta=torch.Tensor(emb.size(-1)),
    learn_alpha=True,
    learn_beta=True,
    learn_threshold=True,
    init_hidden=True
    )
# syn2, membrane_pot2 = synaptic2.init_synaptic()

synaptic3 = snn.Synaptic(
    # alpha=torch.Tensor(emb.size(-1) * 3),
    # beta=torch.Tensor(emb.size(-1) * 3),
    alpha=torch.Tensor(emb.size(-1)),
    beta=torch.Tensor(emb.size(-1)),
    learn_alpha=True,
    learn_beta=True,
    learn_threshold=True,
    init_hidden=True,
    output=True
    )
# syn3, membrane_pot3 = synaptic3.init_synaptic()

In [ ]:
synaptic

In [ ]:
linear = torch.nn.Linear(in_features=emb.size(-1), out_features=emb.size(-1))
# linear2 = torch.nn.Linear(in_features=emb.size(-1) * 2, out_features=emb.size(-1) * 2)
linear2 = torch.nn.Linear(in_features=emb.size(-1), out_features=emb.size(-1))

In [ ]:
from models.layers.SynapticChain import SynapticChain
chain = SynapticChain(hidden_size=hidden_size, return_last=True)
spike, mem_pot, syn_cur = chain(emb)

In [ ]:
assert not torch.isnan(syn_cur).any()

In [ ]:
spikes =[]
mem_pots = []
# flat = emb.flatten(start_dim=2)
for timestep in range(t):
    # spike, syn, membrane_pot = synaptic(emb[:, timestep,:, :], syn, membrane_pot)
    spike = synaptic(emb[:, timestep,:, :])
    l = linear(spike)
    # spike2, syn2, membrane_pot2 = synaptic2(torch.cat((spike, emb[:, timestep,:, :]), dim=2), syn2, membrane_pot2)
    # spike2, syn2, membrane_pot2 = synaptic2(torch.cat((l, emb[:, timestep,:, :]), dim=2), syn2, membrane_pot2)
    spike2 = synaptic2(l)
    l2 = linear2(spike2)
    
    # spike3, syn3, membrane_pot3 = synaptic3(torch.cat((l2, l), dim=2), syn3, membrane_pot3)
    spike3, syn3, membrane_pot3 = synaptic3(l2)
    
    # spike, syn, membrane_pot = rsynaptic(flat[:, timestep, :], spike, syn, membrane_pot)
    # spike, syn, membrane_pot = rsynaptic(emb[:, timestep, :, :], spike, syn, membrane_pot)
    # spike, syn, membrane_pot = rsynaptic_conv(emb[:, timestep, :, :], spike, syn, membrane_pot)
    spikes.append(spike3)
    mem_pots.append(membrane_pot3)
    

In [ ]:
855900

In [ ]:
spike3.size()

In [ ]:
torch.cat((spike, emb[:, 0,:, :]), dim=2).size()

In [ ]:
spike.size(), syn.size(), membrane_pot.size()

In [ ]:
space = DiffConv(in_channels=hidden_size,
                                 out_channels=hidden_size,
                                 k=2)

In [ ]:
stacked_spikes = torch.stack(spikes, 1)
post_space = space(stacked_spikes[:,-1,:,:], batch.edge_index, batch.edge_weight)

In [ ]:
post_space

In [ ]:
decoder = nn.Linear(32, input_size * horizon)
post_decoder = decoder(post_space)

In [ ]:
post_decoder.size()

In [ ]:
from einops.layers.torch import Rearrange
rearrange = Rearrange('b n (t f) -> b t n f', t=horizon)
post_rearange = rearrange(post_decoder)
post_rearange.size()

In [ ]:
post_rearange

In [ ]:
torch.stack(spikes, 1).size(), torch.stack(spikes, 1).reshape((b,t,n,-1)).size()

In [ ]:
spike.reshape((b,n,-1)).size()

In [ ]:
snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True, output=True,threshold=thresh)
                          

In [ ]:
emb.size()

In [ ]:
emb[:, 0, :, :].size()

In [ ]:
spk, mem = rlif(emb[:, 0, :, :], spike, membrane_pot)

In [ ]:
spk.size(), mem.size()

In [ ]:
from models.TemporalSpikeGraphConvNet import TemporalSpikeGraphConvNet
from models.TemporalSynapticGraphConvNet import TemporalSynapticGraphConvNet

model = TemporalSynapticGraphConvNet(input_size=input_size,
                               n_nodes=n_nodes,
                               horizon=horizon,
                               hidden_size=hidden_size * 5,
                               output_type="membrane_potential",
                               number_of_blocks=3
                               )
model

In [ ]:
device

In [ ]:
model.to(device)
batch.to(device)
res = model(*batch.input)

In [ ]:
res.size()

In [ ]:
res